In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import statsmodels.api as sm

## Read in 2003 Data

In [2]:
#Read in files 2003 Consumption
consump_03_01 = pd.read_csv('2003_Consumption/FILE01.csv')
consump_03_01.columns = [col.strip() for col in consump_03_01.columns]
consump_03_01 = consump_03_01[['PUBID8', 'REGION8', 'CENDIV8', 'SQFT8', 'PBA8', 'YRCON8', 'FREESTN8'
                              ,'GLSSPC8', 'NELVTR8', 'NESLTR8', 'OPEN248', 'WKHRS8',
                              'NWKER8']]

consump_03_02 = pd.read_csv('2003_Consumption/FILE02.csv')
consump_03_02.columns = [col.strip() for col in consump_03_02.columns]
consump_03_02 = consump_03_02[['PUBID8', 'ONEACT8']]

consump_03_04 = pd.read_csv('2003_Consumption/FILE04.csv')
consump_03_04.columns = [col.strip() for col in consump_03_04.columns]
consump_03_04 = consump_03_04[['PUBID8','RFGEQP8']]

consump_03_05 = pd.read_csv('2003_Consumption/FILE05.csv')
consump_03_05.columns = [col.strip() for col in consump_03_05.columns]
consump_03_05 = consump_03_05[['PUBID8','ELHT18', 'ELCOOL8','ELWATR8','ELCOOK8','ELMANU8']]

consump_03_15 = pd.read_csv('2003_Consumption/FILE15.csv')
consump_03_15.columns = [col.strip() for col in consump_03_15.columns]
consump_03_15 = consump_03_15[['PUBID8','ELCNS8']]

#Merge Dataframes
dfs = [consump_03_01, consump_03_02, consump_03_04, consump_03_05, consump_03_15]
consump_03 = reduce(lambda left,right: pd.merge(left,right,on='PUBID8'), dfs)

consump_03['YEAR'] = 2003

## Read in 2012 Data

In [3]:
consump_12 = pd.read_csv('2012_public_use_data_aug2016.csv')

consump_12 = consump_12[['PUBID', 'REGION', 'CENDIV', 'SQFT', 'PBA','YRCON','FREESTN', 'NELVTR','NESLTR', 'GLSSPC',
          'OPEN24', 'WKHRS', 'NWKER', 'ONEACT', 'RFGEQP', 'ELHT1', 'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU',
          'ELCNS']]

consump_12['YEAR'] = 2012

There are climate fields PUBCLIM in 2012 and CLIMAT in 2003 that refer to a climate type a building is located.  These are based on the number of heating and cooling days.  We could potentially use this as a consumption feature by tying zipcodes to NOAA data which has the heating/cooling days for each of the stations.

GlassPercent Categories are different, might need to standardize these if we actually use them

## Merge Years & Data Standarization 

In [4]:
for i in consump_03.columns:
    consump_03[i] = pd.to_numeric(consump_03[i], errors = 'coerce')
    
consump_03.columns = list(consump_12.columns)

consump_all = pd.concat([consump_12,consump_03])

In [5]:
PBA_Dict = {
1:'Vacant',
2:'Office',
4:'Laboratory',
5:'Nonrefrigerated warehouse',
6:'Food sales',
7:'Public order and safety',
8:'Outpatient health care',
11:'Refrigerated warehouse',
12:'Religious worship',
13:'Public assembly',
14:'Education',
15:'Food service',
16:'Inpatient health care',
17:'Nursing',
18:'Lodging',
23:'Strip shopping mall',
24:'Enclosed mall',
25:'Retail other than mall',
26:'Service',
91: 'Other'}

In [6]:
pba_list = []
for i in consump_all['PBA']:
    pba_list.append(PBA_Dict[i])

consump_all['PBA_Detail'] = pd.Series(pba_list)

In [7]:
consump_all['NELVTR'] = consump_all['NELVTR'].fillna(value=0)
consump_all['NESLTR'] = consump_all['NESLTR'].fillna(value=0)

In [8]:
#Filter for freestanding building with a single primary activity
consump_filtered = consump_all.loc[(consump_all['FREESTN'] == 1) & (consump_all['ONEACT'] == 1)]
consump_filtered = consump_filtered.dropna(axis=0, how = 'any')

binary_fix = ['OPEN24','RFGEQP', 'ELHT1','ELCOOL','ELWATR', 'ELCOOK', 'ELMANU']
for column in binary_fix:
    consump_filtered[column] = consump_filtered[column].replace(to_replace = 2, value = 0)
    
max_val_fix = ['NELVTR', 'NESLTR']
for column in binary_fix:
    consump_filtered[column] = consump_filtered[column].replace(to_replace = 995, value = 51)

# Regression Modeling 

In [9]:
consump_filtered.columns

Index(['PUBID', 'REGION', 'CENDIV', 'SQFT', 'PBA', 'YRCON', 'FREESTN',
       'NELVTR', 'NESLTR', 'GLSSPC', 'OPEN24', 'WKHRS', 'NWKER', 'ONEACT',
       'RFGEQP', 'ELHT1', 'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU', 'ELCNS',
       'YEAR', 'PBA_Detail'],
      dtype='object')

In [10]:
X = consump_filtered[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU']]
y = consump_filtered['ELCNS']
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.666
Method:                 Least Squares   F-statistic:                     947.4
Date:                Sun, 11 Mar 2018   Prob (F-statistic):               0.00
Time:                        09:57:10   Log-Likelihood:                -95871.
No. Observations:                5687   AIC:                         1.918e+05
Df Residuals:                    5675   BIC:                         1.918e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
SQFT          15.8491      0.409     38.774      0.000        15.048    16.650
WKHRS      -6002.3730   2578.976     -2.327      0.020     -1.11e+04  -946.594
NWKER       3588.2452    180.144     19.919      0.000      3235.094  3941.397
OPEN24      2.611e+06   3.41e+05      7.661      0.000      1.94e+06  3.28e+06
NELVTR      1.673e+04   1634.235     10.237      0.000      1.35e+04  1.99e+04
NESLTR      3926.7169   1849.149      2.124      0.034       301.678  7551.756
RFGEQP     -2.317e+05   1.95e+05     -1.189      0.235     -6.14e+05   1.5e+05
ELHT1        8.06e+04   1.55e+05      0.521      0.603     -2.23e+05  3.84e+05
ELCOOL      2.913e+04   1.95e+05      0.150      0.881     -3.53e+05  4.11e+05
ELWATR     -1.078e+05   1.45e+05     -0.744      0.457     -3.92e+05  1.76e+05
ELCOOK      3.002e+05   1.57e+05      1.906      0.057     -8501.790  6.09e+05
ELMANU      1.907e+05   3.58e+05      0.533      0.594     -5.11e+05  8.92e+05
==============================================================================
Omnibus:                    11496.260   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         60122678.417
Skew:                          16.306   Prob(JB):                         0.00
Kurtosis:                     505.656   Cond. No.                     1.59e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.59e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
consump_filtered.groupby('PBA_Detail').sum()

,PUBID,REGION,CENDIV,SQFT,PBA,YRCON,FREESTN,NELVTR,NESLTR,GLSSPC,...,NWKER,ONEACT,RFGEQP,ELHT1,ELCOOL,ELWATR,ELCOOK,ELMANU,ELCNS,YEAR
PBA_Detail,,,,,,,,,,,,,,,,,,,,,
Education,2418818,1901,3747,61876609,9864,1326281,706.0,410.0,73.0,1844.0,...,47332.0,706.0,601.0,187.0,619.0,242.0,296.0,7.0,7.309274e+08,1420436
Enclosed mall,98057,83,160,26487000,744,61353,31.0,127.0,149.0,59.0,...,17216.0,31.0,31.0,13.0,31.0,24.0,20.0,0.0,4.218073e+08,62372
Food sales,322581,309,614,2936501,668,219478,112.0,4.0,1.0,256.0,...,9081.0,112.0,111.0,54.0,105.0,72.0,51.0,2.0,1.157423e+08,225335
Food service,995125,831,1644,5765652,4484,554339,299.0,22.0,101.0,761.0,...,10112.0,299.0,299.0,108.0,277.0,117.0,155.0,11.0,1.735214e+08,601552
Inpatient health care,1117991,874,1721,198001502,5498,676424,345.0,10024.0,99.0,1081.0,...,388501.0,345.0,344.0,24.0,318.0,59.0,227.0,3.0,6.077738e+09,694104
Laboratory,130022,89,176,6366001,132,63591,33.0,96.0,0.0,109.0,...,10966.0,33.0,33.0,2.0,30.0,8.0,7.0,0.0,2.845264e+08,66396
Lodging,903245,725,1436,46466354,4720,500196,264.0,1752.0,1183.0,772.0,...,26871.0,264.0,251.0,148.0,241.0,66.0,95.0,0.0,7.499353e+08,531150
Nonrefrigerated warehouse,2065561,1728,3472,80234359,3046,1165133,610.0,114.0,147.0,982.0,...,41801.0,610.0,388.0,175.0,465.0,324.0,30.0,76.0,6.153269e+08,1227257
Nursing,275628,212,405,7921250,1462,168408,86.0,152.0,2.0,240.0,...,7291.0,86.0,86.0,37.0,84.0,24.0,48.0,2.0,1.216268e+08,173032


# Business Specific Models

In [71]:
#Office Model

office_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == 'Office']

X = office_df[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU']]
y = office_df['ELCNS']
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     271.4
Date:                Sat, 03 Mar 2018   Prob (F-statistic):          4.45e-322
Time:                        15:11:50   Log-Likelihood:                -18729.
No. Observations:                1139   AIC:                         3.748e+04
Df Residuals:                    1127   BIC:                         3.754e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
SQFT          11.4132      0.752     15.174      0.000         9.937    12.889
WKHRS      -5895.2605   6524.186     -0.904      0.366     -1.87e+04  6905.657
NWKER       1892.8153    283.906      6.667      0.000      1335.772  2449.859
OPEN24      1.933e+06   8.26e+05      2.341      0.019      3.13e+05  3.55e+06
NELVTR      1.708e+04   2241.630      7.619      0.000      1.27e+04  2.15e+04
NESLTR     -1065.5614   3448.050     -0.309      0.757     -7830.881  5699.758
RFGEQP      1.161e+05   2.74e+05      0.423      0.672     -4.22e+05  6.54e+05
ELHT1        4.28e+05   2.25e+05      1.906      0.057     -1.26e+04  8.69e+05
ELCOOL      3.132e+05   3.69e+05      0.848      0.396     -4.11e+05  1.04e+06
ELWATR     -2.957e+05   2.22e+05     -1.333      0.183     -7.31e+05   1.4e+05
ELCOOK      2.494e+05    2.7e+05      0.923      0.356      -2.8e+05  7.79e+05
ELMANU     -2.501e+05    6.5e+05     -0.385      0.700     -1.52e+06  1.02e+06
==============================================================================
Omnibus:                     2098.639   Durbin-Watson:                   1.962
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3509767.843
Skew:                          12.732   Prob(JB):                         0.00
Kurtosis:                     273.751   Cond. No.                     2.75e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.75e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
#Food Service Model

food_serve_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == 'Religious worship']

X = food_serve_df[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU']]
y = food_serve_df['ELCNS']
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                     1593.
Date:                Sun, 11 Mar 2018   Prob (F-statistic):          8.47e-276
Time:                        14:23:02   Log-Likelihood:                -4724.2
No. Observations:                 330   AIC:                             9472.
Df Residuals:                     318   BIC:                             9518.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
SQFT          18.4791      0.443     41.689      0.000        17.607    19.351
WKHRS       -441.1442    947.751     -0.465      0.642     -2305.799  1423.511
NWKER      -7571.5322    259.793    -29.145      0.000     -8082.662 -7060.402
OPEN24     -6.933e+04   1.67e+05     -0.416      0.678     -3.97e+05  2.58e+05
NELVTR      -3.51e+05   3.78e+04     -9.286      0.000     -4.25e+05 -2.77e+05
NESLTR      1.216e+06   4.63e+04     26.238      0.000      1.12e+06  1.31e+06
RFGEQP     -1.007e+05   6.19e+04     -1.628      0.105     -2.22e+05   2.1e+04
ELHT1       3.178e+04   5.39e+04      0.590      0.556     -7.42e+04  1.38e+05
ELCOOL     -9.832e+04   5.95e+04     -1.652      0.099     -2.15e+05  1.87e+04
ELWATR      9.929e+04   4.94e+04      2.008      0.045      2029.128  1.97e+05
ELCOOK      3.447e+04   4.95e+04      0.696      0.487     -6.29e+04  1.32e+05
ELMANU      1.584e+05   2.94e+05      0.539      0.590     -4.19e+05  7.36e+05
==============================================================================
Omnibus:                      133.607   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5732.123
Skew:                          -0.891   Prob(JB):                         0.00
Kurtosis:                      23.340   Cond. No.                     1.74e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.74e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [77]:
#Nonrefrigerated warehouse Model

ware_nofridge_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == 'Nonrefrigerated warehouse']

X = ware_nofridge_df[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU']]
y = ware_nofridge_df['ELCNS']
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     181.6
Date:                Sat, 03 Mar 2018   Prob (F-statistic):          2.31e-190
Time:                        15:17:37   Log-Likelihood:                -9743.4
No. Observations:                 610   AIC:                         1.951e+04
Df Residuals:                     598   BIC:                         1.956e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
SQFT           3.7029      0.469      7.904      0.000         2.783     4.623
WKHRS       1070.2875   2583.374      0.414      0.679     -4003.301  6143.876
NWKER       8703.9733    837.298     10.395      0.000      7059.572  1.03e+04
OPEN24       5.89e+04   4.22e+05      0.139      0.889     -7.71e+05  8.89e+05
NELVTR     -2.892e+05   1.59e+05     -1.820      0.069     -6.01e+05  2.29e+04
NESLTR      2.324e+05   9.48e+04      2.452      0.014      4.63e+04  4.19e+05
RFGEQP       7.26e+04   2.18e+05      0.333      0.739     -3.55e+05  5.01e+05
ELHT1       2.824e+05   2.02e+05      1.400      0.162     -1.14e+05  6.79e+05
ELCOOL     -2.327e+05   2.43e+05     -0.956      0.340     -7.11e+05  2.45e+05
ELWATR     -2.629e+05   1.93e+05     -1.361      0.174     -6.42e+05  1.16e+05
ELCOOK     -1.803e+05   4.14e+05     -0.435      0.663     -9.94e+05  6.33e+05
ELMANU      5.447e+05   2.63e+05      2.071      0.039      2.82e+04  1.06e+06
==============================================================================
Omnibus:                      932.134   Durbin-Watson:                   2.076
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           456497.375
Skew:                           8.361   Prob(JB):                         0.00
Kurtosis:                     135.970   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""